In [ ]:
import pandas as pd
import numpy as np
import copy

from datetime import datetime, timedelta, time

In [ ]:
from solver import Solver

In [ ]:
slvr = Solver('data/flights.json', 'data/crew.json')

slvr.flights

In [ ]:
slvr.crew

In [ ]:
slvr.initial_assignment()

slvr.assignment

In [ ]:
slvr.initial_crew_state()
slvr.crew_state 

In [ ]:
total_assignment_cost, diagnostics = slvr.compute_assignment_cost(slvr.assignment)
print(total_assignment_cost)

In [ ]:
total_crew_cost, diagnostics = slvr.compute_crew_cost(slvr.crew_state)
print(total_crew_cost)

In [ ]:
destroy_repair_strength = 0.2
num_destroy_operators = 2
num_repair_operators = 1

num_to_destroy = int(len(slvr.flights) * destroy_repair_strength)

In [ ]:
#print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}, \n {r_assignment_cost}, {r_crew_cost} => {r_assignment_cost+r_crew_cost}")

In [ ]:
total_assignment_cost, _ = slvr.compute_assignment_cost(slvr.assignment)
total_crew_cost, _ = slvr.compute_crew_cost(slvr.crew_state)

for i in range(1000):

    d_assignment, d_crew_state, flights_destroyed, affected_crew_list = slvr.destroy(num_to_destroy)

    r_assignment, r_crew_state = slvr.repair()
    r_assignment_cost, _ = slvr.compute_assignment_cost(slvr.r_assignment)
    r_crew_cost, _ = slvr.compute_crew_cost(slvr.r_crew_state)

    if (r_assignment_cost+r_crew_cost) < (total_assignment_cost+total_crew_cost):
        slvr.accept_change()
        total_assignment_cost = r_assignment_cost
        total_crew_cost = r_crew_cost
    
        print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}")

print("finished")
print(f"{total_assignment_cost}, {total_crew_cost} => {total_assignment_cost+total_crew_cost}")

In [ ]:
slvr.assignment

In [ ]:
slvr.crew_state